# Analysis of CERN Beam Position Dataset

## 1] Data structure and statistical analysis

Dataset consists of: 33 individual experiments, each experiment has:
1. 2 independent axes of measurement [x and y]
2. 536 individual sensors per axis
3. 6600 measurements per sensor
Totalling 233,481,600 measurement points for analysis.    
    
Data is supplied as 66 comma seperated value files, one per axis and experiment.

Missing information:
1. Frequency - unclear on data frequency capture i.e. rotation/sensor capture rate.
2. Threshold value for beam being detected or not.

In [ ]:
"""
Machine Specifications:
Processor: Intel-i9 10900K; clocked to 4.7 Ghz; watercooled
Mobo: ASUS ROGSTRX Z490-F
Memory: Corsair Vengeance 32GB 3200 MHz DDR4
SSD:  Samsung 1TB 970 EVO
GPU: EVGA NVIDIA GeForce RTX 3080 XC3 BLACK 10GB
OS: Windows 10 [10.0.19042 Build 19042]
"""

#import required librarys for exploratory data analysis
import os
import numpy as np #version 1.19.2
import pandas as pd #version 1.1.3
import time
from itertools import cycle
import scipy.stats as stats
import math

import plotly.express as px
import plotly.io as pio
pio.templates.default = "simple_white"
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from yellowbrick.cluster import SilhouetteVisualizer,KElbowVisualizer

Create a list of files constituting the dataset for easy access:

In [ ]:
FILE_LIST = os.listdir(".\data")

Exmaine the file list:

In [ ]:
for i in range(0,6):
    print(f"File {i+1}: {FILE_LIST[i]}")

Files are structured in pairs, hence read the first and second files which are the x and y axis datasets for the first experiment dated 29May2018 at 17:48pm [and 36.697s].

Data to be recorded in a Pandas DataFrame for interrogation.

In [ ]:
data_read_x = pd.read_csv(f".\data\{FILE_LIST[0]}",delimiter=",")
data_read_y = pd.read_csv(f".\data\{FILE_LIST[1]}",delimiter=",")

Examine the first DataFrame for the X axis.

In [ ]:
data_read_x

Data is configured into records or measurements [time] across the columns and sensor numbers on the rows.

Examine the min, max, mean, standard deviation and 25, 50 and 75% quartile values by event for first ten events:

In [ ]:
data_read_x.iloc[:,0:10].describe()

Examine the min, max, mean, standard deviation and 25, 50 and 75% quartile values by sensor for first ten sensors:

In [ ]:
data_read_x.transpose().iloc[:,0:10].describe()

The min, max and standard deviation values by sensor are much lower than the values by event; indicating that there is more variation sensor to sensor than there is event to event.

The 25% and 75% quartiles by event appear to range between +/- 0.5 the maxium values appear to be below 9.

Create a heatmap of the data at two ranges: +/-0.5 and +/-9.0 to visualise the initial observations.

In [ ]:
def generate_heatmap(input_data, lower_z, upper_z):
    heatmap = px.imshow(
                      input_data,
                      labels=dict(x="Turn [count]",
                                  y="Sensor [number]"),
                      width=1080,height=1080,
                      zmin=lower_z,zmax=upper_z
                     )

    heatmap.update_xaxes(dtick=500)
    heatmap.update_yaxes(dtick=50)

    heatmap.show()

In [ ]:
generate_heatmap(data_read_x,-0.5,0.5)

In [ ]:
generate_heatmap(data_read_y,-0.5,0.5)

In [ ]:
generate_heatmap(data_read_x,-9,9)

In [ ]:
generate_heatmap(data_read_y,-9,9)

find max sensor
find min sensor
plot distributions
plot sensor reading vs. cycle for several close to each other

Transpose the input data so the individual sensors are columns / features:

In [ ]:
data_read_x_T_desc = data_read_x.transpose().describe()
data_read_x_T_desc

Produce a histogram of the means for each sensor:

In [ ]:
histogram_x_sensor_means = px.histogram(data_read_x_T_desc.transpose()[["mean"]],
                                        log_y=True,
                                        title="Distribution of Mean Sensor Readings",
                                        )
histogram_x_sensor_means.update_xaxes(title="Sensor Reading [-]",
                  dtick="5")

histogram_x_sensor_means.update_yaxes(title="Count [-]")

histogram_x_sensor_means.show()

In [ ]:
histogram_x_sensor_std = px.histogram(data_read_x_T_desc.transpose()[["std"]],
                                        log_y=False,
                                        title="Distribution of Sensor Reading Standard Deviations",
                                        )
histogram_x_sensor_std.update_xaxes(title="Sensor Reading [-]",
                  dtick="0.1")

histogram_x_sensor_std.update_yaxes(title="Count [-]")

histogram_x_sensor_std.show()

Using a log scale for the y axis allows outling sensors to be identified.

Transpose the descriptive statistics table so the statistical descriptors [mean, max, ...] are features / columns:

In [ ]:
data_read_x_T_desc_T=data_read_x_T_desc.transpose()
data_read_x_T_desc_T

Determine the minimum and maximum reading sensors:

In [ ]:
print(data_read_x_T_desc_T[data_read_x_T_desc_T["mean"]==data_read_x_T_desc_T["mean"].max()])

In [ ]:
print(data_read_x_T_desc_T[data_read_x_T_desc_T["mean"]==data_read_x_T_desc_T["mean"].min()])

Plot a histogram of the maximum and minimum reading sensors to examine the distributions:

In [ ]:
histogram_x_maximum_sensor = px.histogram(data_read_x.iloc[485,:],
                                          log_y=True,
                                          title="Distribution of Readings for Sensor 485",
                                          nbins=50
                                          )
histogram_x_maximum_sensor.update_xaxes(title="Sensor Reading [-]",
                  dtick="0.1")

histogram_x_maximum_sensor.update_yaxes(title="Count [-]")

histogram_x_maximum_sensor.show()

In [ ]:
histogram_x_minimum_sensor = px.histogram(data_read_x.iloc[145,:],
                                         log_y=True,
                                         title="Distribution of Readings for Sensor 145",
                                         nbins=50
                                         )
histogram_x_minimum_sensor.update_xaxes(title="Sensor Reading [-]",
                  dtick="0.1")

histogram_x_minimum_sensor.update_yaxes(title="Count [-]")

histogram_x_minimum_sensor.show()

In [ ]:
histogram_x_median_sensor = px.histogram(data_read_x.iloc[526,:],
                                          log_y=True,
                                          title="Distribution of Readings for Sensor 526",
                                          nbins=50
                                          )
histogram_x_median_sensor.update_xaxes(title="Sensor Reading [-]",
                  dtick="0.1")

histogram_x_median_sensor.update_yaxes(title="Count [-]")

histogram_x_median_sensor.show()

In [ ]:
histogram_x_zero_sensor = px.histogram(data_read_x.iloc[0,:],
                                          log_y=True,
                                          title="Distribution of Readings for Sensor 0",
                                          nbins=50
                                          )
histogram_x_zero_sensor.update_xaxes(title="Sensor Reading [-]",
                  dtick="0.1")

histogram_x_zero_sensor.update_yaxes(title="Count [-]")

histogram_x_zero_sensor.show()

Four unique distributions observed from four different sensors:
1. Sensor 0 shows a very uniform distribution between -1.5 and 1.5. Potentially a mix of two normal distributions.
2. Sensor 526 shows very little distribution, nearly all points at exactly zero with four individual readings slightly lower.
3. Sensor 145 [minimum mean] 

Examine scatter plots of each sensor vs. rotation to observe if there are any paterns over time.

In [ ]:
sensor_list=[0,145,485,526]

for s in sensor_list:
    line_chart = px.line(data_read_x.iloc[s,:],
                                    title=f"Line Plot for Sensor: {s}")
    line_chart.update_xaxes(range=[0,100],dtick=10)
    line_chart.show()

## Complete Memory Efficiency Analysis for Wide vs. Long Format

In [ ]:
def wide_format_ingest(file_count):

    #create a dataframe to store all the ingested data into
    MASTER_INPUT=pd.DataFrame()
    
    for FILE in FILE_LIST[0:file_count]:
        #print the active file for status check
        print(f"Reading file name...{FILE}")
        FILE_PATH="./data/{}".format(FILE)
        
        #read the file
        RAW_INPUT=pd.read_csv(FILE_PATH,delimiter=",")
        
        #extract the file name for the unique experiment log
        RAW_INPUT["experiment"]=FILE[11:34]
        
        #extract the axis for analysis
        RAW_INPUT["axis"]=FILE[34]
        
        #append each file to the master dataframe
        MASTER_INPUT=MASTER_INPUT.append(RAW_INPUT)

    #reset the index to give each row a unique index number
    MASTER_INPUT=MASTER_INPUT.reset_index()
    
    #swap the old index column to be sensor number
    MASTER_INPUT=MASTER_INPUT.rename({"index":"sensor"},axis=1)
 
    #iterate the old columns for turns to a new column with "turn" in the column name
    ORIGINAL_COL_NAMES = MASTER_INPUT.columns[np.arange(1,6601)]
    NEW_COL_NAMES = ["turn_"+str(i) for i in np.arange(1,len(ORIGINAL_COL_NAMES)+1)]
    COLUMN_MAPPER = dict(zip(ORIGINAL_COL_NAMES,NEW_COL_NAMES))
    MASTER_INPUT=MASTER_INPUT.rename(columns=COLUMN_MAPPER)
    
    #use one-hot encoding to split out the axis column from string to numeric in case required
    MASTER_INPUT=pd.get_dummies(MASTER_INPUT,columns=["axis"])    
    
    #set the datatypes for ease of analysis
    MASTER_INPUT=MASTER_INPUT.astype({"axis_x":float,
                                      "axis_y":float,
                                      "sensor":int
                                     })
    
    return MASTER_INPUT

In [ ]:
def long_format_ingest(maximum_file):
    
    MASTER_INPUT=pd.DataFrame()
    
    for FILE in FILE_LIST[0:maximum_file]:
        print(f"Reading file name...{FILE}")
        FILE_PATH="./data/{}".format(FILE)
        
        #transpose the read dataframe so sensors become columns
        RAW_INPUT=pd.read_csv(FILE_PATH,delimiter=",").transpose()
        
        #melt the dataframe into sensor number and readings
        RAW_INPUT=pd.melt(RAW_INPUT,ignore_index=True,var_name="sensor",value_name="reading")
        
        #create a column to capture the turn number as a feature
        TURNS=cycle(np.arange(0,6600,1))
        RAW_INPUT["turn"]=[next(TURNS) for TURN in range(len(RAW_INPUT))]
        
        RAW_INPUT["experiment"]=FILE[11:34]        
        RAW_INPUT["axis"]=FILE[34]
        MASTER_INPUT=MASTER_INPUT.append(RAW_INPUT)

    #reset index and drop old index
    MASTER_INPUT=MASTER_INPUT.reset_index().drop("index",axis=1)
    
    MASTER_INPUT=pd.get_dummies(MASTER_INPUT,columns=["axis"])
    
    MASTER_INPUT=MASTER_INPUT.astype({"axis_x":float,
                                      "axis_y":float})
    
    return MASTER_INPUT

In [ ]:
wide_format_data_sample=wide_format_ingest(6)
long_format_data_sample=long_format_ingest(6)

In [ ]:
print(f"Wide format shape: {wide_format_data_sample.shape}")
print(f"Long format shape: {long_format_data_sample.shape}")

In [ ]:
wide_format_data_sample.info(memory_usage="deep")

In [ ]:
long_format_data_sample.info(memory_usage="deep")

Wide format is significantly more memory efficint and will be used going forward

## 2] Prototype Algorithm Development

Initially read in ~10% of the dataset to begin algorithm selection on:

In [ ]:
%%time
prototype_sample = wide_format_ingest(6)
prototype_sample.shape

In [ ]:
prototype_sample.info()

In [ ]:
prototype_sample.head()

Split out the specific columns that will be used as features in the algorithm:

In [ ]:
modelling_data=prototype_sample.iloc[:,1:6601]

Create a KMeans model to be used for Elbow analysis in selecting the optimum number of clusters. Random_state 13 has been selected for reproduceability and repeatability and will be used throughout.

In [ ]:
kmeans_model_elbow = KMeans(random_state=13,algorithm="full")

Here "YellowBrick" [https://www.scikit-yb.org/en/latest/] for easy visualisation of the KMeans results.

Initially this will be elbow analysis for k=3 to 12 and determining the optimium number of clusters

In [ ]:
kelbow_visualizer=KElbowVisualizer(
    kmeans_model_elbow,
    k=(3,13)
    )

kelbow_visualizer.fit(modelling_data)

kelbow_visualizer.show()

In [ ]:
kelbow_visualizer_cal_har=KElbowVisualizer(
    kmeans_model_elbow,
    k=(3,13),
    metric="calinski_harabasz"
    )

kelbow_visualizer_cal_har.fit(modelling_data)

kelbow_visualizer_cal_har.show()

The elbow analysis indicates that k=5 will be the optimum; silhouette analysis completed on k=4, 5 and 6:

In [ ]:
cluster_count=[4,5,6]

for i in cluster_count:
    kmeans_model = KMeans(n_clusters=i,algorithm="full",random_state=13)
    visualiser=SilhouetteVisualizer(kmeans_model,colors="yellowbrick")
    
    t0=time.time()
    visualiser.fit(modelling_data)    
    t1=time.time()
    total_time=t1-t0
    visualiser.show()
    print(f"Time taken: {total_time:.1f}s")

Silhouette analysis also concludes that k=5 is the optimum number of clusters.

In depth analysis of k=5 completed in order to understand the cluster distribution.

In [ ]:
kmeans_model = KMeans(n_clusters=5,algorithm="full",random_state=13)
kmeans_model.fit(modelling_data)

#store the cluster labels back in the master dataframe [adding 1 to each label to use 1-5 rather than 0-4]
prototype_sample["cluster"]=kmeans_model.labels_+1

In [ ]:
#create a histrogram of cluster vs. observations
fig = px.histogram(prototype_sample,
                   x="cluster",
                  log_y=True,
                  title=f"Cluster distribution for 5 clusters; across 3 experiments only")
fig.update_layout(bargap=0.1)
fig.update_xaxes(dtick=1,title="Cluster [-]")
fig.update_yaxes(title="Count [-]")
fig.show()

Clusters 2 and 3 appear to have the lowest count; next a summary table was created for each cluster capturing the number of observations, how many unique sensors were in the cluster, the number of observations that were in the X and Y axes, the number of unique experiment ID's and then the cluster mean and standard deviation.

In [ ]:
#create a results dataframe for summary statistics
RESULTS_DF=pd.DataFrame(columns=[
    "cluster_no", #the cluster ID
    "count", #the number of observations in the cluster
    "unique_sensors", #the unique sensor count in the cluster
    "x_axis_count", #the number of observations from the X axis
    "y_axis_count", #the number of observations from the Y axis
    "experiment_count", #the number of unique experiment ID's in the cluster
    "mean", #the cluster mean
    "standard_deviation" #the cluster standard deviation
])

for i in range(1,(prototype_sample.cluster.max()+1)):
    #create a workingdata frame for the cluster being analysed
    working_data=prototype_sample[prototype_sample["cluster"]==i]
    
    #count the number of observations
    count=len(working_data)
    
    #count the unique sensor observations
    unique_sensors=len(working_data.sensor.unique())
    
    #count the number of observations from the X axis
    x_axis_count=len(working_data[working_data["axis_x"]==1])
    
    #count the number of observations from the Y axis
    y_axis_count=len(working_data[working_data["axis_y"]==1])
    
    #count the number of unique experiment ID's
    experiment_count=len(working_data.experiment.unique())

    #convert the cluster obervations [sensors and turns] to a numpy array
    working_data=working_data.iloc[:,1:6601].to_numpy()
    
    #calcuate the mean of the numpy array
    mean=working_data.mean()
    
    #calculate the standard deviation of the numpy array
    std_dev=working_data.std()
    
    #append the results to the results summary dataframe
    RESULTS_DF=RESULTS_DF.append({
        "cluster_no":i,
        "count":count,
        "unique_sensors":unique_sensors,
        "x_axis_count":x_axis_count,
        "y_axis_count":y_axis_count,
        "experiment_count":experiment_count,
        "mean":mean,
        "standard_deviation":std_dev
    },ignore_index=True)
    
#set the data types for each column for readability
RESULTS_DF=RESULTS_DF.astype({
    "cluster_no":int,
    "count":int,
    "unique_sensors":int,
    "x_axis_count":int,
    "y_axis_count":int,
    "experiment_count":int,
})

In [ ]:
RESULTS_DF

The results dataframe shows that two of the clusters only contain one unique sensor, with three instances [one for each experiment] from the X-axis.

Cluster 1 contains the bulk of the observations with a very event split between the x and y axis.

The mean and standard deviation can be used to plot distributions, assuming a normal distribution, to visualise the clusters in the measurement space:

In [ ]:
plt.figure(figsize=(20,10))
plt.grid(b=True,which="major")
plt.xlabel("Sensor Measurement [-]")
plt.ylabel("Probability Density")

for i in RESULTS_DF.cluster_no.unique():
    #retrieve the mean of the cluster being analysed
    mu=RESULTS_DF[RESULTS_DF["cluster_no"]==i].loc[:,"mean"]
    
    #retrieve the standard deviation of the cluster being analysed
    sigma=RESULTS_DF[RESULTS_DF["cluster_no"]==i].loc[:,"standard_deviation"]
    
    #create a series of 1000 equally spaces points on the x-axis to cover +/- standard deviations
    x=np.linspace(mu-3*sigma,mu+3*sigma,1000)
    
    #using the x-values apply a normal distribution for the mean and standard deviation of the cluster
    y=stats.norm.pdf(x,mu,sigma)
    
    #create a lable for the cluster being analysed
    label=f"Cluster {i:.0f}"
    
    #plot the distribution for the cluster
    plt.plot(x,y,label=label)

#include the legend on the plot
plt.legend()

It can be seen there are five distinct clusters. There is likely some overlap between clusters 1, 4 and 5 which is reducing the Silhouette score - but only slightly. However, cluster 2 and 3 are very distinct from the rest.

The next step is to evaluate this on a larger dataset.

## 3] Verification of prototype on ~20% of dataset

In [ ]:
verification_sample = wide_format_ingest(14)

In [ ]:
verification_modelling_data=verification_sample.iloc[:,1:6601]

In [ ]:
kmeans_verification = KMeans(n_clusters=5,algorithm="full",random_state=13)
kmeans_verification.fit(verification_modelling_data)
verification_sample["cluster"]=kmeans_verification.labels_+1

In [ ]:
fig = px.histogram(verification_sample,
                   x="cluster",
                  log_y=True,
                  title=f"Cluster distribution for 5 clusters; across 7 experiments only")
fig.update_layout(bargap=0.1)
fig.update_xaxes(dtick=1,title="Cluster [-]")
fig.update_yaxes(title="Count [-]")
fig.show()

In [ ]:
VERIFICATION_DF=pd.DataFrame(columns=[
    "cluster_no",
    "count",
    "unique_sensors",
    "x_axis_count",
    "y_axis_count",
    "experiment_count",
    "mean",
    "standard_deviation"
])

for i in range(1,(verification_sample.cluster.max()+1)):
    working_data=verification_sample[verification_sample["cluster"]==i]
    count=len(working_data)
    unique_sensors=len(working_data.sensor.unique())
    x_axis_count=len(working_data[working_data["axis_x"]==1])
    y_axis_count=len(working_data[working_data["axis_y"]==1])
    experiment_count=len(working_data.experiment.unique())

    working_data=working_data.iloc[:,1:6601].to_numpy()
    mean=working_data.mean()
    std_dev=working_data.std()
    
    VERIFICATION_DF=VERIFICATION_DF.append({
        "cluster_no":i,
        "count":count,
        "unique_sensors":unique_sensors,
        "x_axis_count":x_axis_count,
        "y_axis_count":y_axis_count,
        "experiment_count":experiment_count,
        "mean":mean,
        "standard_deviation":std_dev
    },ignore_index=True)
    
VERIFICATION_DF=VERIFICATION_DF.astype({
    "cluster_no":int,
    "count":int,
    "unique_sensors":int,
    "x_axis_count":int,
    "y_axis_count":int,
    "experiment_count":int,
})

In [ ]:
plt.figure(figsize=(20,10))
plt.grid(b=True,which="major")
plt.xlabel("Sensor Measurement [-]")
plt.ylabel("Probability Density")

for i in VERIFICATION_DF.cluster_no.unique():
    mu=VERIFICATION_DF[VERIFICATION_DF["cluster_no"]==i].loc[:,"mean"]
    sigma=VERIFICATION_DF[VERIFICATION_DF["cluster_no"]==i].loc[:,"standard_deviation"]
    x=np.linspace(mu-3*sigma,mu+3*sigma,1000)
    y=stats.norm.pdf(x,mu,sigma)
    label=f"Cluster {i:.0f}"
    plt.plot(x,y,label=label)
plt.legend()

It can be see the distributions are still nicely distinct from each other, showing the model and hyperparameters have scaled up nicely to a slightly larger dataset.